In [90]:
import transformers
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
import numpy as np
from tensorflow import Tensor
import pandas as pd
import tensorflow as tf
import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score, accuracy_score

VOCAB_SIZE = 15000
SEQUENCE_LENGTH = 100
EMBED_DIM = 8
SEED = 0

DATA_PATH=r"../../data/transformed/amazon_reviews_5_partition_1.csv"

BERT_MODEL = "distilbert-base-uncased-finetuned-sst-2-english"

In [121]:
model = TFDistilBertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2)
tokenizer = DistilBertTokenizerFast.from_pretrained(BERT_MODEL)


Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_159']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [146]:
inputs = keras.Input(shape=(2,),dtype="int64")
x = keras.layers.Dense(100, activation="relu")(inputs)
outputs = keras.layers.Dense(1)(x)
regressor = keras.Model(inputs=inputs, outputs=outputs)

In [132]:
df_reader = pd.read_csv(
    DATA_PATH,
    index_col=0,
    chunksize=10
)
df = df_reader.__next__()
df = df.dropna()
df["reviewText"] = df["reviewText"].convert_dtypes(convert_string=True)

In [133]:
x = tokenizer(
    df["reviewText"].to_list(),
    truncation=True,
    padding=True,
    return_tensors="tf"
)

y = (df["overall"] > 3) * 1

x=dict(x)

In [110]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [142]:
preds = model.predict(x)["logits"]

1/1 [==============================] - 1s 1s/step


In [147]:
regressor.compile(optimizer="adam", loss="mse", metrics="mae")

In [148]:
regressor.fit(preds, np.array(y), epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 712ms/step - loss: 0.2895 - mae: 0.4524
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 0.2434 - mae: 0.4011
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 0.2066 - mae: 0.3500
Epoch 4/100
1/1 [==============================] - 0s 9ms/step - loss: 0.1791 - mae: 0.3000
Epoch 5/100
1/1 [==============================] - 0s 9ms/step - loss: 0.1605 - mae: 0.2518
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 0.1500 - mae: 0.2400
Epoch 7/100
1/1 [==============================] - 0s 8ms/step - loss: 0.1461 - mae: 0.2462
Epoch 8/100
1/1 [==============================] - 0s 13ms/step - loss: 0.1470 - mae: 0.2583
Epoch 9/100
1/1 [==============================] - 0s 17ms/step - loss: 0.1506 - mae: 0.2756
Epoch 10/100
1/1 [==============================] - 0s 11ms/step - loss: 0.1551 - mae: 0.2944
Epoch 11/100
1/1 [==============================] - 0s 11ms/step - loss: 

In [111]:
inputs = keras.Input(shape=(512,), dtype="int64")
bert = model.layers[0](inputs)
bert = bert[0]
flat = keras.layers.Flatten()(bert)
regressor = keras.layers.Dense(1)(flat)

new_model = keras.Model(inputs=inputs, outputs=regressor)

In [113]:
new_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [76]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    x,
    y.to_list()
))

In [77]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])